In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

/tmp/ipykernel_90265/378180666.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
books = books.drop(['Image-URL-S', 'Image-URL-M',
                   'Image-URL-L', 'Publisher', 'Book-Author', 'Year-Of-Publication'], axis=1)
books.head()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


In [7]:
books_merged = books.merge(ratings, on='ISBN')
books_merged.head(10)

,ISBN,Book-Title,User-ID,Book-Rating
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0
5,0002005018,Clara Callan,67544,8
6,0002005018,Clara Callan,85526,0
7,0002005018,Clara Callan,96054,0
8,0002005018,Clara Callan,116866,9
9,0002005018,Clara Callan,123629,9


In [8]:
combine_book_rating = books_merged.dropna(
    axis=0, subset=['Book-Title'])

book_ratingCount = (combine_book_rating.
                    groupby(by=['Book-Title'])['Book-Rating'].
                    count().
                    reset_index().
                    rename(columns={'Book-Rating': 'totalRatingCount'})
                    [['Book-Title', 'totalRatingCount']]
                    )
book_ratingCount.head()


,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [9]:
book_ratingCount = books_merged.merge(
    book_ratingCount, left_on='Book-Title', right_on='Book-Title', how='left')
book_ratingCount.head()


,ISBN,Book-Title,User-ID,Book-Rating,totalRatingCount
0,0195153448,Classical Mythology,2,0,2
1,0002005018,Clara Callan,8,5,14
2,0002005018,Clara Callan,11400,0,14
3,0002005018,Clara Callan,11676,8,14
4,0002005018,Clara Callan,41385,0,14


In [10]:
popularity_threshold = 25
rating_popular_book = book_ratingCount.query(
    'totalRatingCount >= @popularity_threshold')
rating_popular_book.head()


,ISBN,Book-Title,User-ID,Book-Rating,totalRatingCount
30,0399135782,The Kitchen God's Wife,8,0,311
31,0399135782,The Kitchen God's Wife,11676,9,311
32,0399135782,The Kitchen God's Wife,29526,9,311
33,0399135782,The Kitchen God's Wife,36836,0,311
34,0399135782,The Kitchen God's Wife,46398,9,311


In [11]:
rating_popular_book.head()


,ISBN,Book-Title,User-ID,Book-Rating,totalRatingCount
30,0399135782,The Kitchen God's Wife,8,0,311
31,0399135782,The Kitchen God's Wife,11676,9,311
32,0399135782,The Kitchen God's Wife,29526,9,311
33,0399135782,The Kitchen God's Wife,36836,0,311
34,0399135782,The Kitchen God's Wife,46398,9,311


In [12]:
# Pivot matrix
pivot = rating_popular_book.pivot_table(
    index='Book-Title', columns='User-ID', values='totalRatingCount').fillna(0)

pivot

User-ID,8,9,10,14,16,17,19,23,26,32,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"14,000 Things to Be Happy About",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from scipy.sparse import csr_matrix
pivot_matrix = csr_matrix(pivot.values)

In [14]:
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(pivot_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
query_index = np.random.choice(pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

5117


In [16]:
pivot.index[query_index]


'The Solitaire Mystery'

In [17]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(
            pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(
            i, pivot.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendations for The Solitaire Mystery:

1: Enduring Love, with distance of 0.866592530806986:
2: The Name of the Rose: including Postscript to the Name of the Rose, with distance of 0.8703810381586968:
3: The Unbearable Lightness of Being : A Novel (Perennial Classics), with distance of 0.8754661296533205:
4: A Clockwork Orange (Norton Paperback Fiction), with distance of 0.8754661296533205:
5: Smoke and Mirrors: Short Fictions and Illusions, with distance of 0.8785646518562167:
